<center><h1 style="color:#1a1a1a;
                    font-size:3em">
        NHANES - MTech Project 
        </h1> 
        <h2 style="color:#1a1a1a;
                    font-size:2em">
        Feature Selection - Previous Studies fields - Correlation - Data collection (Non Lab)
       </h2>
       <h3 style="color:#1a1a1a;
                    font-size:2em">
        Prakash Easow Thomas - 2022DA04285 - June 2024
       </h3>
</center>

# 1. Importing Libraries and Reading the Datasets

## 1.1 Importing the Libraries

In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

## 1.2 Reading the NHANES 2018 Datasets

In [5]:
df1lab  = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\laboratory.csv')
df2demo = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\demographics.csv')
df3diet = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\dietary.csv')
df4exam = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\examination.csv')
df5ques = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\questionnaire.csv')

## 1.3 Variables from the Previous Studies (Excluding lab Variables) + HbA1c

In [7]:
columns_to_select = [
'RIDAGEYR',         #	Age in years at screening
'RIDRETH3',         #	Race/Hispanic origin w/ NH Asian
'RIAGENDR',         #	Gender
'PAQ706',           #  Days physically active at least 60 min.
'PAQ605',           #  Vigorous work activity
'PAQ620',           #	Moderate work activity
'PAQ650',           #	Vigorous recreational activities
'PAQ665',           #	Moderate recreational activities
'BMXBMI',           #	Body Mass Index (kg/m**2)
'DIQ175A',          #	Family history
'RHQ162',           #	During pregnancy, told you have diabetes
'DR1TALCO',         #	Alcohol (gm)
'DID260',           #	How often check blood for glucose/sugar
'BPXOSY1',          #	Systolic - 1st oscillometric reading
'BPXODI1',          #	Diastolic - 1st oscillometric reading
'BPXOSY2',          #	Systolic - 2nd oscillometric reading
'BPXODI2',          #	Diastolic - 2nd oscillometric reading
'BPXOSY3',          #	Systolic - 3rd oscillometric reading
'BPXODI3',          #	Diastolic - 3rd oscillometric reading
'MCQ160B',          #	Ever told had congestive heart failure
'DS2TCHOL',         #	Cholesterol (mg)
'DSQTCHOL',         #	Cholesterol (mg)
'DMDEDUC3',         #	Education level - Children/Youth 6-19
'DMDEDUC2',         #	Education level - Adults 20+
'CBD765',           #	Respondent's education level
'BPD035',           #	Age told had hypertension
'BPQ040A',          #	Taking prescription for hypertension
#'INQ020',           #	Income from wages/salaries
#'INQ012',           #	Income from self employment
#'INQ030',           #	Income from Social Security or RR
#'INQ060',           #	Income from other disability pension
#'INQ080',           #	Income from retirement/survivor pension
#'INQ090',           #	Income from Supplemental Security Income
#'INQ132',           #	Income from state/county cash assistance
#'INQ140',           #	Income from interest/dividends or rental
#'INQ150',           #	Income from other sources
'IND235',           #	Monthly family income
'DIQ050',           #	Taking insulin now
'RIDEXPRG',         #	Pregnancy status at exam
'SLD012',           #	Sleep hours - weekdays or workdays
'BMXWAIST',         #	Waist Circumference (cm)
'DIQ070',            #  Take diabetic pills to lower blood sugar
'LBXGH'             #   Glycohemoglobin (%) HbA1c
]


# 2. Merging and Selection of Data - NHANES 2018 Data

## 2.1 Merging Data

In [9]:
common_key = 'SEQN'
# Merge the DataFrames
merged_dfNHAENES2018 = df1lab.merge(df2demo, on=common_key, how='outer')
merged_dfNHAENES2018 = merged_dfNHAENES2018.merge(df3diet, on=common_key, how='outer')
merged_dfNHAENES2018 = merged_dfNHAENES2018.merge(df4exam, on=common_key, how='outer')
merged_dfNHAENES2018 = merged_dfNHAENES2018.merge(df5ques, on=common_key, how='outer')

# Save the merged DataFrame to a new CSV file
merged_dfNHAENES2018.to_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\Before_Full_Feature_Selection_09_Data.csv', index=False)

print("Files merged successfully into 'Before_Full_Feature_Selection_09_Data.csv'")

Files merged successfully into 'Before_Full_Feature_Selection_09_Data.csv'


In [10]:
merged_dfNHAENES2018.head()

,SEQN,URXUMA,URXUMS,URXUCR,URXCRS,URDACT,WTSSGP2Y,SSAGP,WTSA2YR,URXUAS,...,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ190,WHQ200,WHQ030M,WHQ500,WHQ520
0,93703.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93704.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93705.0,3.2,3.2,24.0,2121.6,13.33,NaN,NaN,NaN,NaN,...,150.0,130.0,63.0,170.0,62.0,2.0,NaN,NaN,NaN,NaN
3,93706.0,39.6,39.6,69.0,6099.6,57.39,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,150.0,17.0,2.0,NaN,NaN,NaN,NaN
4,93707.0,41.6,41.6,209.0,18475.6,19.90,NaN,NaN,24984.393699,5.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,2.0


In [11]:
merged_dfNHAENES2018.shape

(9254, 2236)

## 2.2 Selection of Data

In [13]:
valid_columns = pd.Index(columns_to_select).intersection(merged_dfNHAENES2018.columns)
merged_dfNHAENES2018_selected = merged_dfNHAENES2018[columns_to_select]
merged_dfNHAENES2018_selected.shape

(9254, 34)

In [14]:
nan_counts = merged_dfNHAENES2018_selected.isna().sum()
print("\nCount of NaN values in each column:")
#print(nan_counts)
print(type(nan_counts))


Count of NaN values in each column:
<class 'pandas.core.series.Series'>


In [15]:
sorted_series = nan_counts.sort_values()
print(sorted_series)

RIDAGEYR       0
RIDRETH3       0
RIAGENDR       0
DIQ050       357
IND235       653
BMXBMI      1249
BMXWAIST    1653
DR1TALCO    1770
BPXODI1     3111
BPXOSY1     3111
BPXOSY2     3131
BPXODI2     3131
SLD012      3141
BPXOSY3     3160
BPXODI3     3160
LBXGH       3209
PAQ665      3398
PAQ650      3398
PAQ605      3398
PAQ620      3398
MCQ160B     3685
DMDEDUC2    3685
PAQ706      6503
DMDEDUC3    6948
BPD035      7117
BPQ040A     7117
RHQ162      7141
DIQ070      7580
CBD765      7598
DIQ175A     8109
RIDEXPRG    8144
DID260      8361
DSQTCHOL    8821
DS2TCHOL    8970
dtype: int64


# 3. Data Preprocessing - NHANES 2018 Data
         - Remove the Non-Numeric Fields 
         - Remove the records where (LBXGH) No HbA1c is not present  
         - Initilized NaN with zeros   

## 3.1  Removing the Non-Numeric Data

In [17]:
def clean_data(df):
    for column in df.columns:
        # Check if the column contains byte-strings
        if df[column].dtype == object:
            # Decode byte-strings to regular strings
            df[column] = df[column].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
        # Convert non-numeric values to NaN for numeric columns
        df[column] = pd.to_numeric(df[column], errors='ignore')
    return df

In [18]:

merged_dfNHAENES2018_Cleaned =clean_data(merged_dfNHAENES2018_selected)
print(merged_dfNHAENES2018_Cleaned)

      RIDAGEYR  RIDRETH3  RIAGENDR  PAQ706  PAQ605  PAQ620  PAQ650  PAQ665  \
0          2.0       6.0       2.0     7.0     NaN     NaN     NaN     NaN   
1          2.0       3.0       1.0     7.0     NaN     NaN     NaN     NaN   
2         66.0       4.0       2.0     NaN     2.0     2.0     2.0     1.0   
3         18.0       6.0       1.0     NaN     2.0     2.0     2.0     1.0   
4         13.0       7.0       1.0     0.0     NaN     NaN     NaN     NaN   
...        ...       ...       ...     ...     ...     ...     ...     ...   
9249       0.0       2.0       1.0     NaN     NaN     NaN     NaN     NaN   
9250      11.0       3.0       2.0     7.0     NaN     NaN     NaN     NaN   
9251      77.0       3.0       2.0     NaN     2.0     2.0     2.0     2.0   
9252       0.0       3.0       2.0     NaN     NaN     NaN     NaN     NaN   
9253      64.0       4.0       1.0     NaN     2.0     2.0     2.0     2.0   

      BMXBMI  DIQ175A  ...  CBD765  BPD035  BPQ040A  IND235  DI

C:\Users\praka\AppData\Local\Temp\ipykernel_53960\3940003188.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_numeric(df[column], errors='ignore')


## 3.2  Removing the records where HbA1c(LBXGH) is not present

In [20]:
merged_dfNHAENES2018_Cleaned_numeric = merged_dfNHAENES2018_Cleaned.select_dtypes(include=[int, float])
column_to_check_HbA1c ='LBXGH'
merged_dfNHAENES2018_Cleaned_numeric_HBV =merged_dfNHAENES2018_Cleaned_numeric.dropna(subset=[column_to_check_HbA1c])
column_to_check_BMI = 'BMXBMI'
merged_dfNHAENES2018_Cleaned_numeric_HBV =merged_dfNHAENES2018_Cleaned_numeric_HBV.dropna(subset=[column_to_check_BMI])

merged_dfNHAENES2018_Cleaned_numeric_HBV =merged_dfNHAENES2018_Cleaned_numeric_HBV.reset_index(drop=True)

#dataset_for_ML = dataset_for_ML.reset_index(drop=True)
print(merged_dfNHAENES2018_Cleaned_numeric_HBV)

      RIDAGEYR  RIDRETH3  RIAGENDR  PAQ706  PAQ605  PAQ620  PAQ650  PAQ665  \
0         66.0       4.0       2.0     NaN     2.0     2.0     2.0     1.0   
1         18.0       6.0       1.0     NaN     2.0     2.0     2.0     1.0   
2         13.0       7.0       1.0     0.0     NaN     NaN     NaN     NaN   
3         66.0       6.0       2.0     NaN     2.0     2.0     2.0     1.0   
4         75.0       4.0       2.0     NaN     2.0     1.0     2.0     2.0   
...        ...       ...       ...     ...     ...     ...     ...     ...   
5946      70.0       6.0       2.0     NaN     2.0     2.0     2.0     1.0   
5947      42.0       1.0       1.0     NaN     1.0     1.0     2.0     2.0   
5948      41.0       4.0       2.0     NaN     2.0     2.0     2.0     1.0   
5949      14.0       4.0       2.0     4.0     NaN     NaN     NaN     NaN   
5950      38.0       3.0       1.0     NaN     2.0     2.0     2.0     2.0   

      BMXBMI  DIQ175A  ...  CBD765  BPD035  BPQ040A  IND235  DI

In [21]:
nan_counts = merged_dfNHAENES2018_Cleaned_numeric_HBV.isna().sum()
print("\nCount of NaN values in each column:")
print(nan_counts)


Count of NaN values in each column:
RIDAGEYR       0
RIDRETH3       0
RIAGENDR       0
PAQ706      5187
PAQ605       776
PAQ620       776
PAQ650       776
PAQ665       776
BMXBMI         0
DIQ175A     4904
RHQ162      3930
DR1TALCO     486
DID260      5158
BPXOSY1      674
BPXODI1      674
BPXOSY2      685
BPXODI2      685
BPXOSY3      701
BPXODI3      701
MCQ160B     1011
DS2TCHOL    5678
DSQTCHOL    5557
DMDEDUC3    4940
DMDEDUC2    1011
CBD765      5564
BPD035      4057
BPQ040A     4057
IND235       375
DIQ050         0
RIDEXPRG    4956
SLD012       537
BMXWAIST     213
DIQ070      4440
LBXGH          0
dtype: int64


In [22]:
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['PAQ706'].isna(), 'PAQ706'] = 0
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['PAQ605'].isna(), 'PAQ605'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['PAQ620'].isna(), 'PAQ620'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['PAQ650'].isna(), 'PAQ650'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['PAQ665'].isna(), 'PAQ665'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['MCQ160B'].isna(), 'MCQ160B'] = 2
#merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['INQ020'].isna(), 'INQ020'] = 2
#merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['INQ012'].isna(), 'INQ012'] = 2
#merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['INQ030'].isna(), 'INQ030'] = 2
#merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['INQ060'].isna(), 'INQ060'] = 2
#merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['INQ080'].isna(), 'INQ080'] = 2
#merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['INQ090'].isna(), 'INQ090'] = 2
#merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['INQ132'].isna(), 'INQ132'] = 2
#merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['INQ140'].isna(), 'INQ140'] = 2
#merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['INQ150'].isna(), 'INQ150'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['BPQ040A'].isna(), 'BPQ040A'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['DIQ050'].isna(), 'DIQ050'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['DIQ070'].isna(), 'DIQ070'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['RIDEXPRG'].isna(), 'RIDEXPRG'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['DIQ175A'].isna(), 'DIQ175A'] = 2  
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['RHQ162'].isna(), 'RHQ162'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['SLD012'].isna(), 'SLD012'] = 8
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['BMXWAIST'].isna(), 'BMXWAIST'] = 34
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['DS2TCHOL'].isna(), 'DS2TCHOL'] = 160
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['DSQTCHOL'].isna(), 'DSQTCHOL'] = 160
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['DSQTCHOL'].isna(), 'DSQTCHOL'] = 160
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['DID260'].isna(), 'DID260'] = 0
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['BPD035'].isna(), 'BPD035'] = 0
merged_dfNHAENES2018_Cleaned_numeric_HBV['DR1TALCO'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['DR1TALCO'].min(), inplace=True)
#merged_dfNHAENES2018_Cleaned_numeric_HBV['DR1TALCO'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['DR1TALCO'].avg(), inplace=True)
merged_dfNHAENES2018_Cleaned_numeric_HBV['IND235'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['IND235'].min(), inplace=True)

In [23]:
nan_counts = merged_dfNHAENES2018_Cleaned_numeric_HBV.isna().sum()
print("\nCount of NaN values in each column:")
print(nan_counts)


Count of NaN values in each column:
RIDAGEYR       0
RIDRETH3       0
RIAGENDR       0
PAQ706         0
PAQ605         0
PAQ620         0
PAQ650         0
PAQ665         0
BMXBMI         0
DIQ175A        0
RHQ162         0
DR1TALCO       0
DID260         0
BPXOSY1      674
BPXODI1      674
BPXOSY2      685
BPXODI2      685
BPXOSY3      701
BPXODI3      701
MCQ160B        0
DS2TCHOL       0
DSQTCHOL       0
DMDEDUC3    4940
DMDEDUC2    1011
CBD765      5564
BPD035         0
BPQ040A        0
IND235         0
DIQ050         0
RIDEXPRG       0
SLD012         0
BMXWAIST       0
DIQ070         0
LBXGH          0
dtype: int64


In [24]:
def fill_77_99_Monthly_family_income(row,field):
    if row[field] == 77  or row[field] == 99:
        row[field] = 0
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['IND235'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_77_99_Monthly_family_income, axis=1, field='IND235')


In [25]:
def fill_999_DID260(row,field):
    if row[field] == 999:
        row[field] = 0
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['DID260'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_999_DID260, axis=1, field='DID260')

In [26]:
def fill_99_DIQ175A(row,field):
    if row[field] == 99:
        row[field] = 0
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['DIQ175A'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_99_DIQ175A, axis=1, field='DIQ175A')

In [27]:
def fill_9_RHQ162(row,field):
    if row[field] == 9:
        row[field] = 0
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['RHQ162'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_9_RHQ162, axis=1, field='RHQ162')

In [28]:
def fill_999_DID260(row,field):
    if row[field] == 999:
        row[field] = 0
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['DID260'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_999_DID260, axis=1, field='DID260')

In [29]:
def fill_9_MCQ160B(row,field):
    if row[field] == 999:
        row[field] = 0
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['MCQ160B'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_9_MCQ160B, axis=1, field='MCQ160B')

In [30]:
def fill_9_DIQ070(row,field):
    if row[field] == 9:
        row[field] = 0
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['DIQ070'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_9_DIQ070, axis=1, field='DIQ070')

In [31]:
def fill_9_DIQ050(row,field):
    if row[field] == 9:
        row[field] = 0
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['DIQ050'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_9_DIQ050, axis=1, field='DIQ050')

In [32]:
def fill_999_BPD035(row,field):
    if row[field] == 999:
        row[field] = 0
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['BPD035'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_999_BPD035, axis=1, field='BPD035')

In [33]:
def fill_na_for_Diastolic_value(row,field):
    if pd.isna(row[field]) and row['RIDAGEYR'] <= 30:
        row[field] = 80
        return row[field] 
    elif pd.isna(row[field]):
        row[field] = 80
        return row[field]  
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['BPXODI1'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_Diastolic_value, axis=1, field='BPXODI1')
merged_dfNHAENES2018_Cleaned_numeric_HBV['BPXODI2'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_Diastolic_value, axis=1, field='BPXODI2')
merged_dfNHAENES2018_Cleaned_numeric_HBV['BPXODI3'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_Diastolic_value, axis=1, field='BPXODI3')

In [34]:
def fill_na_for_Systolic_value(row,field):
    if pd.isna(row[field]) and row['RIDAGEYR'] <= 30:
        row[field] = 120
        return row[field] 
    elif pd.isna(row[field]):
        row[field] = 120
        return row[field]  
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['BPXOSY1'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_Systolic_value, axis=1, field='BPXOSY1')
merged_dfNHAENES2018_Cleaned_numeric_HBV['BPXOSY2'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_Systolic_value, axis=1, field='BPXOSY2')
merged_dfNHAENES2018_Cleaned_numeric_HBV['BPXOSY3'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_Systolic_value, axis=1, field='BPXOSY3')

In [35]:
def fill_na_for_DMDEDUC2_value(row,field): # Adult
    if pd.isna(row[field]) and row['RIDAGEYR'] <= 19 and (row['DMDEDUC3'] <= 9 or row['DMDEDUC3'] == 55 or row['DMDEDUC3']== 66):
        row[field] = 1
        return row[field] 
    elif pd.isna(row[field]) and row['RIDAGEYR'] <= 19 and row['DMDEDUC3'] > 9 and row['DMDEDUC3'] < 13:
        row[field] = 2
        return row[field]  
    elif pd.isna(row[field]) and row['RIDAGEYR'] <= 19 and row['DMDEDUC3'] >= 13 and row['DMDEDUC3'] <= 15:
        row[field] = 3
        return row[field]  
    elif (row[field] == 7 or  row[field] == 9):
        row[field] = 1
        return row[field] 
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['DMDEDUC2'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_DMDEDUC2_value, axis=1, field='DMDEDUC2')

In [36]:
def fill_na_for_DMDEDUC3_value(row,field): # kids
    if pd.isna(row[field])   and row['RIDAGEYR'] > 19 and row['DMDEDUC2'] == 1:
        row[field] = 8
        return row[field] 
    elif pd.isna(row[field]) and row['RIDAGEYR'] > 19 and row['DMDEDUC2'] == 2:
        row[field] = 13
        return row[field]  
    elif pd.isna(row[field]) and row['RIDAGEYR'] > 19 and row['DMDEDUC2'] == 3:
        row[field] = 14
        return row[field]  
    elif pd.isna(row[field]) and row['RIDAGEYR'] > 19 and (row['DMDEDUC2'] == 4 or row['DMDEDUC2'] == 5):
        row[field] = 15
        return row[field]  
#    elif pd.isna(row[field]):
#         print(row['SEQN'])
#        row[field] = 0
#        return row[field]  
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['DMDEDUC3'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_DMDEDUC3_value, axis=1, field='DMDEDUC3')

In [37]:
def fill_na_for_CBD765_value(row,field):
    # print(row[field])
    if pd.isna(row[field]) and (row['DMDEDUC3'] < 13 or row['DMDEDUC3'] == 55 or row['DMDEDUC3']== 66 or row['DMDEDUC2'] < 3):
        row[field] = 1
        return row[field] 
    elif pd.isna(row[field]) and (row['DMDEDUC3'] == 13 or row['DMDEDUC3'] == 14 or row['DMDEDUC2'] == 3):
        row[field] = 2
        return row[field]  
    elif pd.isna(row[field]) and (row['DMDEDUC2'] == 4 or row['DMDEDUC2'] == 5):
        row[field] = 3
        return row[field]  
    else:
        return row[field]
merged_dfNHAENES2018_Cleaned_numeric_HBV['CBD765'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_CBD765_value, axis=1, field='CBD765')

In [38]:
# Processing Physically Active cases
def setting_Physical_Activity(row):    
    if row['PAQ706'] >= 3 and row['PAQ706']  < 7 and  row['RIDAGEYR'] <= 17:
        return 1                                      
    elif row['PAQ706']  == 7 and  row['RIDAGEYR'] <= 17:
        return 2 # if more then set to 2
    elif (row['PAQ620']  == 1  or row['PAQ665'] == 1) and  row['RIDAGEYR'] > 17 :
        return 1 
    elif (row['PAQ605'] == 1 or row['PAQ650'] == 1) and  row['RIDAGEYR'] > 17:
        return 2 # if more then set to 2
    else:
        return 0
merged_dfNHAENES2018_Cleaned_numeric_HBV['Physically_Active_Classify']  = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(lambda row: setting_Physical_Activity(row), axis=1)
merged_dfNHAENES2018_Cleaned_numeric_HBV.shape

(5951, 35)

In [39]:
# Classifying HbA1c Level
def classify(row):
    if row['DIQ070'] == 1 or row['DIQ050'] == 1:
        return 1                 # 2 diabetic 
    elif row['LBXGH']   < 5.7 :
        return 0
    elif row ['LBXGH'] >= 5.7 and row['LBXGH'] < 6.5 :
        return 1
    else:
        return 1             # 2 Diabetic
#print(dataset_for_ML['LBXGH'])
merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXGH_Classify']=  merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(lambda row: classify(row), axis=1)


In [40]:
print(merged_dfNHAENES2018_Cleaned_numeric_HBV)

      RIDAGEYR  RIDRETH3  RIAGENDR  PAQ706  PAQ605  PAQ620  PAQ650  PAQ665  \
0         66.0       4.0       2.0     0.0     2.0     2.0     2.0     1.0   
1         18.0       6.0       1.0     0.0     2.0     2.0     2.0     1.0   
2         13.0       7.0       1.0     0.0     2.0     2.0     2.0     2.0   
3         66.0       6.0       2.0     0.0     2.0     2.0     2.0     1.0   
4         75.0       4.0       2.0     0.0     2.0     1.0     2.0     2.0   
...        ...       ...       ...     ...     ...     ...     ...     ...   
5946      70.0       6.0       2.0     0.0     2.0     2.0     2.0     1.0   
5947      42.0       1.0       1.0     0.0     1.0     1.0     2.0     2.0   
5948      41.0       4.0       2.0     0.0     2.0     2.0     2.0     1.0   
5949      14.0       4.0       2.0     4.0     2.0     2.0     2.0     2.0   
5950      38.0       3.0       1.0     0.0     2.0     2.0     2.0     2.0   

      BMXBMI  DIQ175A  ...  BPQ040A  IND235  DIQ050  RIDEXPRG  

In [41]:
nan_counts = merged_dfNHAENES2018_Cleaned_numeric_HBV.isna().sum()
print("\nCount of NaN values in each column:")
print(nan_counts)


Count of NaN values in each column:
RIDAGEYR                      0
RIDRETH3                      0
RIAGENDR                      0
PAQ706                        0
PAQ605                        0
PAQ620                        0
PAQ650                        0
PAQ665                        0
BMXBMI                        0
DIQ175A                       0
RHQ162                        0
DR1TALCO                      0
DID260                        0
BPXOSY1                       0
BPXODI1                       0
BPXOSY2                       0
BPXODI2                       0
BPXOSY3                       0
BPXODI3                       0
MCQ160B                       0
DS2TCHOL                      0
DSQTCHOL                      0
DMDEDUC3                      0
DMDEDUC2                      0
CBD765                        0
BPD035                        0
BPQ040A                       0
IND235                        0
DIQ050                        0
RIDEXPRG                      0
SLD

In [42]:
columns_to_drop = [
'PAQ706',
'PAQ605',
'PAQ620',
'PAQ650',
'PAQ665',
'LBXGH'
]
merged_dfNHAENES2018_Cleaned_numeric_HBV=merged_dfNHAENES2018_Cleaned_numeric_HBV.drop(columns_to_drop,axis=1)

In [43]:
merged_dfNHAENES2018_Cleaned_numeric_HBV.to_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\After_DataPreprocessing_NoLab.csv', index=False)